In [7]:
import sys
sys.path.append('../')

from autogpt.config import Config
from autogpt.commands.command import command
import json
import requests
import yaml

from duckduckgo_search import ddg


CFG = Config()

## NAVER MAP API
https://developers.naver.com/docs/serviceapi/search/movie/movie.md#%EC%98%81%ED%99%94

In [10]:
import json
import urllib
from urllib.request import Request, urlopen

def get_lng_lat_by_address(address):
    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret 
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(address)
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200):
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        print(response_body)

        if response_body['meta']['totalCount'] == 1 : 
            lng = response_body['addresses'][0]['x']
            lat = response_body['addresses'][0]['y']
            
            return (lng, lat)
        else:
            print('address does not exist')
        
    else:
        print(f'address: {address} Error Code: {res}')


In [21]:
start = '강'
goal = '역삼역'
waypoint = ''

start = get_lng_lat_by_address(start)
end = get_lng_lat_by_address(goal)
# waypoint = get_lng_lat_by_address(waypoint)

{'status': 'OK', 'meta': {'totalCount': 0, 'count': 0}, 'addresses': [], 'errorMessage': ''}
address does not exist
{'status': 'OK', 'meta': {'totalCount': 0, 'count': 0}, 'addresses': [], 'errorMessage': ''}
address does not exist


In [18]:
def get_optimal_route_5(start, goal, waypoints=None, option=''):
    assert start, 'start must be a tuple of (lng, lat)'
    assert goal, 'goal must be a tuple of (lng, lat)'

    assert len(start) == 2, 'start must be a tuple of (lng, lat)'
    assert len(goal) == 2, 'goal must be a tuple of (lng, lat)'

    waypoints = '|'.join([f"{point[0]},{point[1]}" for point in waypoints]) if waypoints else ''
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-5/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&waypoints={waypoints}&option={option}"

    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if (rescode == 200):
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
    else:
        print(f'start: {start} goal: {goal}, waypoints: {waypoints}, Error Code: {rescode}')
        return None

def get_optimal_route_15(start, goal, waypoints=None, option=''):
    assert start, 'start must be a tuple of (lng, lat)'
    assert goal, 'goal must be a tuple of (lng, lat)'

    assert len(start) == 2, 'start must be a tuple of (lng, lat)'
    assert len(goal) == 2, 'goal must be a tuple of (lng, lat)'

    waypoints = '|'.join([f"{point[0]},{point[1]}" for point in waypoints]) if waypoints else ''
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&waypoints={waypoints}&option={option}"

    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if (rescode == 200):
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
    else:
        print(f'start: {start} goal: {goal}, waypoints: {waypoints}, Error Code: {rescode}')
        return None

In [19]:
start

In [17]:

get_optimal_route_5(start, goal)


AssertionError: start must be a tuple of (lng, lat)